In [116]:
from simulator import Simulator, string_to_micro, micro_to_time

In [16]:
class AverageMidAlgo(object):   
    def __init__(self, session, date, tickers):
        # save the session information
        self.session = session
        self.date = date
        self.ticker = tickers[0]  # we only expect one ticker here so just save the one
        
        # set any global variables
        self.start_time = string_to_micro("9:30")
        self.end_time = string_to_micro("16:00")
        self.interval = string_to_micro("1s")
        self.sum_mid = 0.0
        self.num_mid = 0
        
        # subscribe to the ticker of interest, and set the first timer
        self.session.subscribe_ticker_all_feeds(self.ticker)
        self.session.add_timer(self.start_time, self.timer_callback)
        
    def timer_callback(self, time):
        # get the best bid and offer, compute the midmarket, and update averages
        bid, ask = self.session.get_inside_market(self.ticker)
        mid = (bid['price'] + ask['price']) / 2.0 / 1000000.0
        self.sum_mid += mid
        self.num_mid += 1
        ### HERE IS THE NEW LINE:
        #print "time:", micro_to_time(time), "midmarket:", mid, "running average:", self.sum_mid/self.num_mid 
        
        # reset the timer unless we are done
        if time < self.end_time:
            self.session.add_timer(time + self.interval, self.timer_callback) 
    
    def end(self):
        return self.sum_mid / self.num_mid

sim = Simulator(AverageMidAlgo)
sim.run("20170413", ["AAPL"])

Process Process-14:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/share/simulator/current/src/simulator.py", line 45, in runsim
    algo = self.Algo(session, date, tickers, **kwargs)
  File "<ipython-input-16-ef5e35c62f3b>", line 17, in __init__
    self.session.add_timer(self.start_time, self.start_callback)
AttributeError: 'AverageMidAlgo' object has no attribute 'start_callback'


In [136]:
import sys
from simulator import (
    Simulator, string_to_micro, micro_to_time,
    BUY, SELL, SHORT, EXCH_INET,
    BOOK_DEPTH1_PRICE, ORDER_EVENTS,
    )
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.ar_model import AR


class Peggy(object):
    def __init__(self, session, date, tickers, start_time, end_time, side, size):
        self.session = session
        self.date = date
        self.tickers = tickers
        self.ticker1 = self.tickers[0]
        self.ticker2 = self.tickers[1]
        self.start_time = start_time
        self.end_time = end_time
        self.interval = string_to_micro("1s")
        self.state = 'NULL'
        self.side1 = side
        self.side2 = side
        self.order_size = size
        self.buy_shares1 = 0
        self.buy_dollars = 0
        self.sell_shares1 = 0
        self.sell_dollars = 0
        self.buy_shares2 = 0
        self.sell_shares2 = 0
        self.open_longs = 0
        self.open_shorts = 0
        self.close_longs = 0
        self.close_shorts = 0
        
        self.runs = 0
        
        self.dt = 1
        self.long_open = 1.25
        self.long_close = 0.50
        self.short_open = 1.25;
        self.short_close = 0.75
        self.training_size = 100
        
        self.session.add_timer(self.start_time, self.start_callback)
        
        self.results = {'time': []}
        for ticker in self.tickers:
            self.results[ticker] = []
            self.results['return {}'.format(ticker)] = []
        
        # subscribe to the ticker of interest, and set the first timer
        for ticker in self.tickers:
            self.session.subscribe_ticker_all_feeds(ticker)
    
    def start_callback(self, time):
        for ticker in self.tickers:
            self.session.subscribe_event(ticker, ORDER_EVENTS, self.event_callback)
        self.session.add_timer(time, self.timer_callback)
        # reset the timer unless we are done 
        if time < self.end_time:
            self.session.add_timer(time + self.interval, self.start_callback) 
        
    
    def event_callback(self, ticker, event_params):
        self.process_executions(event_params)
        
    def timer_callback(self, time):
        # get the best bid and offer, compute the midmarket, and update averages
        #self.process_executions(event_params)
        self.runs += 1
        self.results['time'].append(micro_to_time(time))
        bid1, ask1 = self.session.get_inside_market(self.ticker1)
        bid2, ask2 = self.session.get_inside_market(self.ticker2)
        
        self.results[self.ticker1].append(self.get_midmarket(self.ticker1) / 1000000.0)
        self.results[self.ticker2].append(self.get_midmarket(self.ticker2) / 1000000.0)
        
        if time > self.start_time + 10**6:
            self.results['return {}'.format(self.ticker1)].append(np.float(self.returns(self.results[self.ticker1][-2:])))
            self.results['return {}'.format(self.ticker2)].append(np.float(self.returns(self.results[self.ticker2][-2:])))
        
        if time > self.start_time + self.training_size * 10**6:   
            returns1 = self.results['return {}'.format(self.ticker1)][-100:]
            returns2 = self.results['return {}'.format(self.ticker2)][-100:]
            residuals, a,b = self.regress(returns1,returns2)
            
            kappa, m, sigma, sigmaeq = self.fitOU(residuals)
            
            s = self.sscore(m, sigmaeq)
            pos = self.buy_shares1 - self.sell_shares1
            self.order_size = 1
            
            # add feature to check if we have orders at the market before we open a position
            orders = self.session.get_all_orders()
        
            if not orders:
                if pos == 0:
                    if s < -self.long_open:
                        self.side1 = BUY
                        self.side2 = SELL
                        price1 = ask1['price']
                        price2 = bid2['price']
                        self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                        self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                        self.open_longs += 1
                        print("open long")
                    elif s > self.short_open:
                        self.side1 = SELL
                        self.side2 = BUY
                        price1 = bid1['price']
                        price2 = ask2['price']
                        self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                        self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                        self.open_shorts += 1
                        print("open short")
                elif pos < 0 and s < self.short_close:
                    self.side1 = BUY
                    self.side2 = SELL
                    price1 = ask1['price']
                    price2 = bid2['price']
                    self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                    self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                    self.close_shorts += 1
                    print("short close")
                elif pos > 0 and s > -self.long_close:
                    self.side1 = SELL
                    self.side2 = BUY
                    price1 = bid1['price']
                    price2 = ask2['price']
                    self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                    self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                    self.close_longs += 1
                    print("long close")
                
            
    def process_executions(self, evp):
        if 'executed_orders' in evp:
            time = self.session.current_time()
            for ex in evp['executed_orders']:
                order = ex['order']
                side = order['side']
                ticker = order['ticker']
                if ticker == self.ticker1:
                    if side == 'B':
                        self.buy_shares1 += ex['quantity_executed']
                        self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                    else:
                        self.sell_shares1 += ex['quantity_executed']
                        self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                    pos = self.buy_shares1 - self.sell_shares1
                elif ticker == self.ticker2:
                    if side == 'B':
                        self.buy_shares2 += ex['quantity_executed']
                        self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                    else:
                        self.sell_shares2 += ex['quantity_executed']
                        self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                    pos = self.buy_shares2 - self.sell_shares2        
                pnl = self.get_pnl()
                print "{0} {1} {quantity_executed} {price_executed} {liquidity} {2} {3}".format(time, side, pos, pnl, **ex)
                

    def get_midmarket(self, ticker):
        bid, ask = self.session.get_inside_market(ticker)
        return (bid['price'] + ask['price']) / 2

    
    def get_pnl(self):
        # mark to mid
        mid1 = self.get_midmarket(self.ticker1)
        mid2 = self.get_midmarket(self.ticker2)
        pnl = self.sell_dollars - self.buy_dollars + (self.buy_shares1 - self.sell_shares1) * mid1 + (self.buy_shares2 - self.sell_shares2) * mid2
        return pnl
    
    def regress(self, returns1,returns2):
        x = np.asarray(returns1).reshape(-1,1)
        y = np.asarray(returns2).reshape(-1,1)
        model = LinearRegression()
        model.fit(x,y)
        a = model.intercept_[0]
        b = model.coef_[0,0]
        residuals = y-model.predict(x)
        return residuals, a,b
    
    def returns(self, midprices):
        return np.diff(midprices, axis=-1)/midprices[:-1]
    
    def fitOU(self, residual):
        ou = np.cumsum(residual)
        model = AR(ou)
        fittedmodel = model.fit(maxlag=1, disp=-1)  
        a = fittedmodel.params[0]
        b = fittedmodel.params[1]
        var =  fittedmodel.sigma2
        kappa = -np.log(b)/self.dt
        m = a/(1-np.exp(-kappa*self.dt))
        sigma = np.sqrt(var*2*kappa/(1-np.exp(-2*kappa*self.dt)))
        sigmaeq = np.sqrt(var/(1-np.exp(-2*kappa*self.dt)));
        return kappa, m, sigma, sigmaeq
    
    def sscore(self, m, sigmaeq):
        if sigmaeq != 0:
            return -m/sigmaeq
        elif m>0:
            return 10000000
        else:
            return -10000000
    
    def end(self):
        print("Total long opens: " + str(self.open_longs))
        print("Total short opens: " + str(self.open_shorts))
        print("Total long closes: " + str(self.close_longs))
        print("Total short closes: " + str(self.close_shorts))
        print('Runs: ' + str(self.runs))
        return self.get_pnl()

date = "20170413"
tickers = ['AAPL', 'MSFT']
start_time = string_to_micro("9:30")
end_time = string_to_micro("9:35")
side = BUY
size = 100
sim = Simulator(Peggy)
sim.run(date, tickers, use_om=True, start_time=start_time, end_time=end_time, side=side, size=size)


open short
34303000248 B 1 65080000 R 1 -5000
34303000248 B 1 65080000 R 2 -10000
34303000248 B 1 65080000 R 3 -15000
34303000248 B 1 65080000 R 4 -20000
34303000248 B 1 65080000 R 5 -25000
34303000248 B 1 65080000 R 6 -30000
34303000248 B 1 65080000 R 7 -35000
34303000248 B 1 65080000 R 8 -40000
34303000248 B 1 65080000 R 9 -45000
34303000248 B 1 65080000 R 10 -50000
34303000248 B 1 65080000 R 11 -55000
34303000248 B 1 65080000 R 12 -60000
34303000248 B 1 65080000 R 13 -65000
34303000248 B 1 65080000 R 14 -70000
34303000248 B 1 65080000 R 15 -75000
34303000248 B 1 65080000 R 16 -80000
34303000248 B 1 65080000 R 17 -85000
34303000248 B 1 65080000 R 18 -90000
34303000248 B 1 65080000 R 19 -95000
34303000248 B 1 65080000 R 20 -100000
34303000248 B 1 65080000 R 21 -105000
34303000248 B 1 65080000 R 22 -110000
34303000248 B 1 65080000 R 23 -115000
34303000248 B 1 65080000 R 24 -120000
34303000248 B 1 65080000 R 25 -125000
34303000248 B 1 65080000 R 26 -130000
34303000248 B 1 65080000 R 27 

34344000248 S 1 65220000 R 103 25995000
34344000248 S 1 65220000 R 102 25990000
34344000248 S 1 65220000 R 101 25985000
34344000248 S 1 65220000 R 100 25980000
34344000248 S 1 65220000 R 99 25975000
34344000248 S 1 65220000 R 98 25970000
34344000248 S 1 65220000 R 97 25965000
34344000248 S 1 65220000 R 96 25960000
34344000248 S 1 65220000 R 95 25955000
34344000248 S 1 65220000 R 94 25950000
34344000248 S 1 65220000 R 93 25945000
34344000248 S 1 65220000 R 92 25940000
34344000248 S 1 65220000 R 91 25935000
34344000248 S 1 65220000 R 90 25930000
34344000248 S 1 65220000 R 89 25925000
34344000248 S 1 65220000 R 88 25920000
34344000248 S 1 65220000 R 87 25915000
34344000248 S 1 65220000 R 86 25910000
34344000248 S 1 65220000 R 85 25905000
34344000248 S 1 65220000 R 84 25900000
34344000248 S 1 65220000 R 83 25895000
34344000248 S 1 65220000 R 82 25890000
34344000248 S 1 65220000 R 81 25885000
34344000248 S 1 65220000 R 80 25880000
34344000248 S 1 65220000 R 79 25875000
34344000248 S 1 65220

34344000248 B 1 141780000 R -41 24330000
34344000248 B 1 141780000 R -40 24315000
34344000248 B 1 141780000 R -39 24300000
34344000248 B 1 141780000 R -38 24285000
34344000248 B 1 141780000 R -37 24270000
34344000248 B 1 141780000 R -36 24255000
34344000248 B 1 141780000 R -35 24240000
34344000248 B 1 141780000 R -34 24225000
34344000248 B 1 141780000 R -33 24210000
34344000248 B 1 141780000 R -32 24195000
34344000248 B 1 141780000 R -31 24180000
34344000248 B 1 141780000 R -30 24165000
34344000248 B 1 141780000 R -29 24150000
34344000248 B 1 141780000 R -28 24135000
34344000248 B 1 141780000 R -27 24120000
34344000248 B 1 141780000 R -26 24105000
34344000248 B 1 141780000 R -25 24090000
34344000248 B 1 141780000 R -24 24075000
34344000248 B 1 141780000 R -23 24060000
34344000248 B 1 141780000 R -22 24045000
34344000248 B 1 141780000 R -21 24030000
34344000248 B 1 141780000 R -20 24015000
34344000248 B 1 141780000 R -19 24000000
34344000248 B 1 141780000 R -18 23985000
34344000248 B 1 

34345000248 B 1 65230000 R 84 22475000
34345000248 B 1 65230000 R 85 22470000
34345000248 B 1 65230000 R 86 22465000
34345000248 B 1 65230000 R 87 22460000
34345000248 B 1 65230000 R 88 22455000
34345000248 B 1 65230000 R 89 22450000
34345000248 B 1 65230000 R 90 22445000
34345000248 B 1 65230000 R 91 22440000
34345000248 B 1 65230000 R 92 22435000
34345000248 B 1 65230000 R 93 22430000
34345000248 B 1 65230000 R 94 22425000
34345000248 B 1 65230000 R 95 22420000
34345000248 B 1 65230000 R 96 22415000
34345000248 B 1 65230000 R 97 22410000
34345000248 B 1 65230000 R 98 22405000
34345000248 B 1 65230000 R 99 22400000
34345000248 B 1 65230000 R 100 22395000
34345000248 B 1 65230000 R 101 22390000
34345000248 B 1 65230000 R 102 22385000
34345000248 B 1 65230000 R 103 22380000
34345000248 B 1 65230000 R 104 22375000
34345000248 B 1 65230000 R 105 22370000
34345000248 S 1 141750000 R 40 22355000
34345000248 S 1 141750000 R 39 22340000
34345000248 S 1 141750000 R 38 22325000
34345000248 S 1 

34360000248 S 1 65280000 R 68 23145000
34360000248 S 1 65280000 R 67 23140000
34360000248 S 1 65280000 R 66 23135000
34360000248 S 1 65280000 R 65 23130000
34360000248 S 1 65280000 R 64 23125000
34360000248 S 1 65280000 R 63 23120000
34360000248 S 1 65280000 R 62 23115000
34360000248 S 1 65280000 R 61 23110000
34360000248 S 1 65280000 R 60 23105000
34360000248 S 1 65280000 R 59 23100000
34360000248 S 1 65280000 R 58 23095000
34360000248 S 1 65280000 R 57 23090000
34360000248 S 1 65280000 R 56 23085000
34360000248 S 1 65280000 R 55 23080000
34360000248 S 1 65280000 R 54 23075000
34360000248 S 1 65280000 R 53 23070000
34360000248 S 1 65280000 R 52 23065000
34360000248 S 1 65280000 R 51 23060000
34360000248 S 1 65280000 R 50 23055000
34360000248 S 1 65280000 R 49 23050000
34360000248 S 1 65280000 R 48 23045000
34360000248 S 1 65280000 R 47 23040000
34360000248 S 1 65280000 R 46 23035000
34360000248 S 1 65280000 R 45 23030000
34360000248 S 1 65280000 R 44 23025000
34360000248 S 1 65280000 

...

34481000248 B 1 141690000 R 64 -60270000
34481000248 B 1 141690000 R 65 -60275000
34481000248 B 1 141690000 R 66 -60280000
34481000248 B 1 141690000 R 67 -60285000
34481000248 B 1 141690000 R 68 -60290000
34481000248 B 1 141690000 R 69 -60295000
34481000248 B 1 141690000 R 70 -60300000
34481000248 B 1 141690000 R 71 -60305000
34481000248 B 1 141690000 R 72 -60310000
34481000248 B 1 141690000 R 73 -60315000
34481000248 B 1 141690000 R 74 -60320000
34481000248 B 1 141690000 R 75 -60325000
34481000248 B 1 141690000 R 76 -60330000
34481000248 B 1 141690000 R 77 -60335000
34481000248 B 1 141690000 R 78 -60340000
34481000248 B 1 141690000 R 79 -60345000
34481000248 B 1 141690000 R 80 -60350000
34481000248 B 1 141690000 R 81 -60355000
34481000248 B 1 141690000 R 82 -60360000
34481000248 B 1 141690000 R 83 -60365000
34481000248 B 1 141690000 R 84 -60370000
34481000248 B 1 141690000 R 85 -60375000
34481000248 B 1 141690000 R 86 -60380000
34481000248 B 1 141690000 R 87 -60385000
34481000248 B 1 

34482000248 B 1 65220000 R 78 -62930000
34482000248 B 1 65220000 R 79 -62935000
34482000248 B 1 65220000 R 80 -62940000
34482000248 B 1 65220000 R 81 -62945000
34482000248 B 1 65220000 R 82 -62950000
34482000248 B 1 65220000 R 83 -62955000
34482000248 B 1 65220000 R 84 -62960000
34482000248 B 1 65220000 R 85 -62965000
34482000248 B 1 65220000 R 86 -62970000
34482000248 B 1 65220000 R 87 -62975000
34482000248 B 1 65220000 R 88 -62980000
34482000248 B 1 65220000 R 89 -62985000
34482000248 B 1 65220000 R 90 -62990000
34482000248 B 1 65220000 R 91 -62995000
34482000248 B 1 65220000 R 92 -63000000
34482000248 B 1 65220000 R 93 -63005000
34482000248 B 1 65220000 R 94 -63010000
34482000248 B 1 65220000 R 95 -63015000
34482000248 B 1 65220000 R 96 -63020000
34482000248 B 1 65220000 R 97 -63025000
34482000248 B 1 65220000 R 98 -63030000
34482000248 B 1 65220000 R 99 -63035000
34482000248 B 1 65220000 R 100 -63040000
34482000248 B 1 65220000 R 101 -63045000
34482000248 B 1 65220000 R 102 -630500

34482096847 S 1 141670000 A 36 -63385000
34482096847 S 1 141670000 A 35 -63390000
34482096847 S 1 141670000 A 34 -63395000
34482096847 S 1 141670000 A 33 -63400000
34482096847 S 1 141670000 A 32 -63405000
34482096847 S 1 141670000 A 31 -63410000
34482096847 S 1 141670000 A 30 -63415000
34482096847 S 1 141670000 A 29 -63420000
34482096847 S 1 141670000 A 28 -63425000
34482096847 S 1 141670000 A 27 -63430000
34482096847 S 1 141670000 A 26 -63435000
34482096847 S 1 141670000 A 25 -63440000
34482096847 S 1 141670000 A 24 -63445000
34482096847 S 1 141670000 A 23 -63450000
34482096847 S 1 141670000 A 22 -63455000
34482096847 S 1 141670000 A 21 -63460000
34482096847 S 1 141670000 A 20 -63465000
34482096847 S 1 141670000 A 19 -63470000
34482096847 S 1 141670000 A 18 -63475000
34482096847 S 1 141670000 A 17 -63480000
34482096847 S 1 141670000 A 16 -63485000
34482096847 S 1 141670000 A 15 -63490000
34482096847 S 1 141670000 A 14 -63495000
34482096847 S 1 141670000 A 13 -63500000
34482096847 S 1 

34482096847 S 1 141670000 A -159 -64360000
34482096847 S 1 141670000 A -160 -64365000
34482096847 S 1 141670000 A -161 -64370000
34482096847 S 1 141670000 A -162 -64375000
34482096847 S 1 141670000 A -163 -64380000
34482096847 S 1 141670000 A -164 -64385000
34482096847 S 1 141670000 A -165 -64390000
34482096847 S 1 141670000 A -166 -64395000
34482096847 S 1 141670000 A -167 -64400000
34482096847 S 1 141670000 A -168 -64405000
34482096847 S 1 141670000 A -169 -64410000
34482096847 S 1 141670000 A -170 -64415000
34482096847 S 1 141670000 A -171 -64420000
34482096847 S 1 141670000 A -172 -64425000
34482096847 S 1 141670000 A -173 -64430000
Total long opens: 0
Total short opens: 1
Total long closes: 22
Total short closes: 22
Runs: 301


11890440000

In [126]:
print(micro_to_time(34482096847))

09:34:42.096847


In [112]:
import sys
from simulator import (
    Simulator, string_to_micro, micro_to_time,
    BUY, SELL, SHORT, EXCH_INET,
    BOOK_DEPTH1_PRICE, ORDER_EVENTS,
    )

class Peggy(object):
    def __init__(self, session, date, tickers, start_time, end_time, side, size):
        self.session = session
        self.date = date
        self.ticker = tickers[0]
        self.start_time = start_time
        self.end_time = end_time

        self.state = 'NULL'
        self.side = side
        self.order_size = size
        self.buy_shares = 0
        self.buy_dollars = 0
        self.sell_shares = 0
        self.sell_dollars = 0
        
        self.session.add_timer(self.start_time, self.start_callback)
        self.session.subscribe_ticker_all_feeds(self.ticker)
 
    def start_callback(self, time):
        self.session.subscribe_event(ticker, BOOK_DEPTH1_PRICE + ORDER_EVENTS, self.event_callback)

    def get_midmarket(self, ticker):
        bid, ask = self.session.get_inside_market(ticker)
        return (bid['price'] + ask['price']) / 2

    def get_pnl(self):
        # mark to mid
        mid = self.get_midmarket(self.ticker)
        pnl = self.sell_dollars - self.buy_dollars + (self.buy_shares - self.sell_shares) * mid
        return pnl

    def process_executions(self, evp):
        if 'executed_orders' in evp:
            time = self.session.current_time()
            for ex in evp['executed_orders']:
                order = ex['order']
                side = order['side']
                ticker = order['ticker']
                if side == 'B':
                    self.buy_shares += ex['quantity_executed']
                    self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                else:
                    self.sell_shares += ex['quantity_executed']
                    self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                pos = self.buy_shares - self.sell_shares
                pnl = self.get_pnl()
                print "{0} {1} {quantity_executed} {price_executed} {liquidity} {2} {3}".format(time, side, pos, pnl, **ex)
    
    
    def event_callback(self, ticker, event_params):
        self.process_executions(event_params)
        bid, ask = self.session.get_inside_market(ticker)
        pos = self.buy_shares - self.sell_shares
        if self.buy_shares == 0:
            self.side = BUY
            self.order_size = 100
        elif pos < 0:
            self.side = BUY
            self.order_size = 200
        else:
            self.side = SELL
            self.order_size = 200
        price = bid['price'] if side == BUY else ask['price']
        orders = self.session.get_orders_by_ticker(self.ticker)

        if not orders:
            self.session.add_order(self.ticker, self.side, self.order_size, price, exchange=EXCH_INET)
        else:
            assert len(orders) == 1
            order = orders[0]
            if order['price'] != price:
                self.session.cancel_order(order['order_id'])
    
    def end(self):
        return self.get_pnl()

date = "20150121"
ticker = "IVV"
start_time = string_to_micro("9:30")
end_time = string_to_micro("9:30:31.000000")
side = BUY
size = 100
sim = Simulator(Peggy)
sim.run(date, [ticker], use_om=True, start_time=start_time, end_time=end_time, side=side, size=size)

IVV
34200062196 B 100 202920000 A 100 500000
IVV
34200062397 S 100 202910000 R 0 -1000000
IVV
34200154996 S 100 202880000 A -100 -1500000
IVV
34200155196 S 100 202880000 A -200 -6000000
IVV
34200213396 B 100 202920000 A -100 -9000000
IVV
34200227796 B 100 202920000 A 0 -9000000
IVV
34200286596 S 100 202910000 R -100 -9000000
IVV
34200400796 B 60 202930000 A -40 -11200000
IVV
34200401796 B 100 202930000 A 60 -10700000
IVV
34200440196 B 40 202930000 A 100 -11500000
IVV
34200463196 S 200 202900000 A -100 -15000000
IVV
34202848799 B 100 202920000 A 0 -16000000
IVV
34203097196 B 100 202920000 A 100 -16500000
IVV
34203097404 S 200 202910000 R -100 -17500000
IVV
34203101396 B 100 202910000 A 0 -17000000
IVV
34203101396 B 100 202910000 A 100 -17000000
IVV
34203221598 S 200 202870000 R -100 -23000000
IVV
34203407796 B 100 202900000 A 0 -24000000
IVV
34203434196 B 100 202900000 A 100 -23500000
IVV
34203519196 S 7 202900000 A 93 -24000000
IVV
34203628196 S 100 202900000 A -7 -24035000
IVV
3420362

IVV
34429593996 S 100 202520000 A -190 -373450000
IVV
34429596396 B 100 202530000 A -90 -374850000
IVV
34429598796 B 100 202530000 A 10 -374450000
IVV
34429598997 S 100 202520000 R -90 -374500000
IVV
34431026396 B 100 202500000 A 10 -372650000
IVV
34431027396 B 100 202500000 A 110 -374350000
IVV
34431027603 S 200 202470000 R -90 -377350000
IVV
34436456596 B 200 202470000 A 110 -376550000
IVV
34436456996 S 200 202460000 A -90 -377550000
IVV
34437847996 B 200 202470000 A 110 -378550000
IVV
34437853397 S 200 202430000 R -90 -383750000
IVV
34437897996 B 100 202460000 A 10 -385200000
IVV
34437897996 B 100 202460000 A 110 -386200000
IVV
34437898200 S 100 202440000 R 10 -387300000
IVV
34437898396 S 100 202440000 A -90 -388200000
IVV
34439522596 B 200 202440000 A 110 -388950000
IVV
34439525197 S 200 202400000 R -90 -393500000
IVV
34448388596 B 200 202510000 A 110 -402700000
IVV
34448422996 S 100 202470000 A 10 -405750000
IVV
34448422996 S 100 202470000 A -90 -408250000
IVV
34459302396 B 100 20

IVV
34583585996 B 200 203060000 A 10 -728200000
IVV
34583774597 S 200 203010000 R -190 -733350000
IVV
34586688196 B 200 203060000 A 10 -738200000
IVV
34586767797 S 100 203040000 A -90 -738300000
IVV
34590794196 B 100 203120000 A 10 -745400000
IVV
34590799196 B 100 203120000 A 110 -747150000
IVV
34590801796 S 200 203060000 R -90 -754800000
IVV
34591084596 B 100 203110000 A 10 -756550000
IVV
34591164596 B 100 203110000 A 110 -757700000
IVV
34591171197 S 100 203070000 R 10 -761000000
IVV
34591189796 S 200 203080000 A -190 -763750000
IVV
34599799396 B 200 203220000 A 10 -787750000
IVV
34599803598 S 200 203160000 R -190 -790000000
IVV
34600554196 B 200 203160000 A 10 -788200000
IVV
34600559396 S 200 203110000 R -190 -793350000
IVV
34601597796 B 200 203140000 A 10 -794500000
IVV
34601918396 S 200 203080000 R -190 -798700000
IVV
34603594596 B 200 203110000 A 10 -800750000
IVV
34603597396 S 200 203060000 R -190 -805850000
IVV
34605932796 B 100 203080000 A -90 -805350000
IVV
34605932996 B 100 2

IVV
34868395196 B 100 204170000 A 14 -1688270000
IVV
34868395397 S 200 204160000 R -186 -1689270000
IVV
34872218596 B 6 204190000 A -180 -1694820000
IVV
34872320196 B 194 204190000 A 14 -1694270000
IVV
34872807996 S 200 204110000 R -186 -1699690000
IVV
34886076796 B 100 204200000 A -86 -1711780000
IVV
34886773396 B 200 204210000 A 114 -1716060000
IVV
34887003396 S 200 204120000 A -86 -1726340000
IVV
34893132796 B 100 204140000 A 14 -1724620000
IVV
34893671396 B 100 204140000 A 114 -1726900000
IVV
34893683396 S 200 204080000 R -86 -1733610000
IVV
34896268196 B 200 204090000 A 114 -1734600000
IVV
34896271397 S 200 204030000 R -86 -1741310000
IVV
34898450996 B 200 204060000 A 114 -1743450000
IVV
34898466996 S 200 204010000 A -86 -1749160000
IVV
34904554196 B 200 204030000 A 114 -1751440000
IVV
34904555997 S 200 203980000 R -86 -1757010000
IVV
34905445796 B 200 204010000 A 114 -1757440000
IVV
34905451796 S 200 203980000 R -86 -1762150000
IVV
34906938596 B 200 204010000 A 114 -1765720000
IV

IVV
35032991396 S 100 203500000 A -186 -2147510000
IVV
35032998996 B 100 203500000 A -86 -2147440000
IVV
35033047396 B 100 203500000 A 14 -2146580000
IVV
35033048796 S 100 203490000 A -86 -2146720000
IVV
35033190796 B 100 203510000 A 14 -2148370000
IVV
35034070396 S 200 203520000 R -186 -2149230000
IVV
35037676996 B 100 203690000 A -86 -2180350000
IVV
35038128996 B 200 203700000 A 114 -2183060000
IVV
35038143996 S 200 203670000 A -86 -2184630000
IVV
35038759396 B 200 203670000 A 114 -2185340000
IVV
35039002796 S 200 203670000 A -86 -2185490000
IVV
35041779996 B 100 203710000 A 14 -2188060000
IVV
35041779996 B 100 203710000 A 114 -2191060000
IVV
35041794396 S 200 203670000 A -86 -2193060000
IVV
35042051598 B 100 203680000 A 14 -2193130000
IVV
35042051598 B 100 203680000 A 114 -2193630000
IVV
35042073996 S 200 203670000 A -86 -2195060000
IVV
35045677196 B 100 203680000 A 14 -2195060000
IVV
35045686196 B 100 203680000 A 114 -2197340000
IVV
35045706196 S 200 203630000 A -86 -2202480000
IVV

35207309996 S 200 203390000 A -86 -2715270000
IVV
35212872796 B 200 203500000 A 114 -2724860000
IVV
35213086996 S 200 203470000 A -86 -2727290000
IVV
35213417196 B 200 203470000 A 114 -2727140000
IVV
35216837796 S 200 203420000 A -86 -2732560000
IVV
35222157796 B 200 203510000 A 114 -2742290000
IVV
35222167397 S 200 203470000 R -86 -2745290000
IVV
35225464596 B 200 203500000 A 114 -2748290000
IVV
35225585996 S 200 203470000 A -86 -2751290000
IVV
35227879196 B 200 203480000 A 114 -2753140000
IVV
35228122796 S 200 203440000 A -86 -2756280000
IVV
35230216396 B 200 203450000 A 114 -2757990000
IVV
35230260996 S 200 203420000 A -86 -2760560000
IVV
35232807796 B 200 203430000 A 114 -2762840000
IVV
35234063196 S 200 203370000 A -86 -2768690000
IVV
35235017596 B 200 203380000 A 114 -2770540000
IVV
35235021597 S 200 203330000 R -86 -2775680000
IVV
35235827596 B 200 203340000 A 114 -2775960000
IVV
35237014596 S 200 203290000 A -86 -2782240000
IVV
35238588596 B 200 203330000 A 114 -2786240000
IVV


IVV
35372844196 S 200 203250000 A -123 -3211830000
IVV
35374341396 B 200 203250000 A 77 -3211370000
IVV
35374347396 S 200 203230000 A -123 -3213370000
IVV
35379147596 B 200 203280000 A 77 -3219445000
IVV
35379321596 S 200 203250000 A -123 -3221830000
IVV
35379769196 B 200 203260000 A 77 -3223370000
IVV
35379876996 S 200 203220000 A -123 -3226755000
IVV
35380233396 B 200 203240000 A 77 -3228910000
IVV
35380237596 S 200 203200000 A -123 -3232295000
IVV
35381313396 B 200 203200000 A 77 -3231220000
IVV
35381318396 S 200 203160000 R -123 -3235990000
IVV
35383549796 B 200 203180000 A 77 -3236760000
IVV
35383553396 S 200 203150000 A -123 -3240145000
IVV
35385109396 B 200 203200000 A 77 -3245220000
IVV
35385501996 S 200 203160000 A -123 -3250605000
IVV
35387365796 B 200 203230000 A 77 -3257295000
IVV
35387440996 S 200 203190000 A -123 -3261680000
IVV
35390200596 B 200 203230000 A 77 -3265295000
IVV
35390273196 S 200 203180000 R -123 -3270450000
IVV
35392755396 B 200 203200000 A 77 -3271220000


35692543796 B 200 203480000 A 177 -3823545000
IVV
35692543997 S 100 203460000 R 77 -3824815000
IVV
35693310396 S 200 203400000 R -123 -3831510000
IVV
35695648396 B 100 203440000 A -23 -3834085000
IVV
35695648596 B 100 203440000 A 77 -3834355000
IVV
35695658396 S 200 203410000 A -123 -3839355000
IVV
35695833996 B 200 203430000 A 77 -3840280000
IVV
35695837196 S 200 203390000 R -123 -3843665000
IVV
35704282596 B 141 203580000 A 18 -3865100000
IVV
35709003796 S 200 203600000 A -182 -3866650000
IVV
35711844596 B 200 203650000 A 18 -3874110000
IVV
35711844798 S 100 203630000 R -82 -3874290000
IVV
35714003396 B 100 203650000 A 18 -3875930000
IVV
35714006196 B 100 203650000 A 118 -3877110000
IVV
35714016396 S 200 203600000 R -82 -3883470000
IVV
35726977996 B 200 203680000 A 118 -3890160000
IVV
35727003396 S 200 203640000 A -82 -3894340000
IVV
35729513396 B 200 203680000 A 118 -3898160000
IVV
35730901996 S 200 203640000 A -82 -3902750000
IVV
35737343196 B 200 203680000 A 118 -3906160000
IVV
35

Process Process-108:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/share/simulator/current/src/simulator.py", line 47, in runsim
    session.run()
  File "<ipython-input-112-0a7a31fa9a4b>", line 59, in event_callback
    def event_callback(self, ticker, event_params):
KeyboardInterrupt


KeyboardInterrupt: 